# Non Quantized

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import time

In [2]:
model_name = "databricks/dolly-v2-3b"

if torch.cuda.is_available():
    device=torch.device("cuda")
else:
    device=torch.device("cpu")

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          cache_dir="cache",
                                          use_flash_attention2=False,
                                          trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/450 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

In [6]:
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             torch_dtype=torch.bfloat16,
                                             trust_remote_code=True,
                                             cache_dir="cache").to(device)

config.json:   0%|          | 0.00/819 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/5.68G [00:00<?, ?B/s]

In [18]:
prompt = '''
You are an expert Chess Playing LLM. Your job is to look at the previous moves made by the other color and then make a move. If there is no move made by the other color, make an opening move.
Your main goal is to try and win the game.
Keep the following instructions  in mind.


## Instructions ##
1. Do not make any illegal chess moves.
2. Try to win, if not possible do not make any moves that are not present in the rules of chess.
3. Do not make up any rules of your own.
4. The "Other Color Move List" will contain the moves made by the other color if any.
5. "Color" will tell you which color of the chess board pieces you belong to - "Black" or "White".
6. "My Move List" will contain all the moves you have made so far.
7. After making one more move in the "My Move List", stop immediately!

Color:
The color you belong to is Black.


##
Other Color Move List :
1. Pawn E4

##

##
My Move List :

##
'''

In [ ]:
a = time.time()
inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)
outputs = model.generate(inputs, max_new_tokens=1024)
print("Time taken is {}s".format(time.time()-a))

print(tokenizer.decode(outputs[0]))